In [1]:
# Advent of Code 2022 - Day 14
# Moideen Kalladi

# open and read input file
with open("input.txt", "r") as f:
    lines = f.read().split("\n")

In [2]:
# parse lines into lists of points
rock_points = []

for line in lines:
    if line == "":
        continue
    line_points = line.split(' -> ')
    points_to_add = []
    for line_point in line_points:
        points_to_add.append(eval(line_point))
    rock_points.append(points_to_add)

rock_points[-3:]

[[(463, 51),
  (463, 53),
  (459, 53),
  (459, 59),
  (470, 59),
  (470, 53),
  (468, 53),
  (468, 51)],
 [(463, 51),
  (463, 53),
  (459, 53),
  (459, 59),
  (470, 59),
  (470, 53),
  (468, 53),
  (468, 51)],
 [(463, 51),
  (463, 53),
  (459, 53),
  (459, 59),
  (470, 59),
  (470, 53),
  (468, 53),
  (468, 51)]]

In [3]:
# why are the last three lines duplicated? hmmm...

In [5]:
lowest_rock = 0
leftmost_rock = 1000
rightmost_rock = -1000

for rp in rock_points:
    for lp in rp:
        if lp[0] < leftmost_rock:
            leftmost_rock = lp[0]
        if lp[0] > rightmost_rock:
            rightmost_rock = lp[0]
        if lp[1] > lowest_rock:
            lowest_rock = lp[1]

print(f"lowest rock: {lowest_rock}")
print(f"leftmost rock: {leftmost_rock}")
print(f"rightmost rock: {rightmost_rock}")

lowest rock: 166
leftmost rock: 409
rightmost rock: 518


In [8]:
# we'll make a grid to model the cave

import numpy as np
import pandas as pd

In [13]:
grid = pd.DataFrame(np.zeros(shape=(167,111), dtype=int), index=list(range(167)), columns=list(range(408,519)))
grid.head()

,408,409,410,411,412,413,414,415,416,417,...,509,510,511,512,513,514,515,516,517,518
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
grid.tail()

,408,409,410,411,412,413,414,415,416,417,...,509,510,511,512,513,514,515,516,517,518
162,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# now draw the rock lines -- set them to 1
for rp in rock_points:
    for i in range(len(rp) - 1):
        point_a = rp[i]
        point_b = rp[i+1]
        
        # if the "right" index (column numbers) are same
        if point_a[0] == point_b[0]:
            start_point = min(point_a[1], point_b[1])
            end_point = max(point_a[1], point_b[1])
            grid[point_a[0]].loc[start_point:end_point] = 1
        
        # if the "down" index (row numbers) are same
        if point_a[1] == point_b[1]:
            start_point = min(point_a[0], point_b[0])
            end_point = max(point_a[0], point_b[0])
            grid.loc[point_a[1]].loc[start_point:end_point] = 1

grid.sum()

408    0
409    2
410    1
411    1
412    1
      ..
514    2
515    1
516    1
517    1
518    1
Length: 111, dtype: int64

In [16]:
grid.tail()

,408,409,410,411,412,413,414,415,416,417,...,509,510,511,512,513,514,515,516,517,518
162,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# simulate sand falling
master_exit = False
sand_rest_counter = 0

while not master_exit:
    sand_rest = False
    current_position = (500,0)
    while not sand_rest:
        # check if reached bottom
        if current_position[1] == 166:
            master_exit = True
            break
        # check directly below
        elif grid[current_position[0]].loc[current_position[1]+1] == 0:
            current_position = (current_position[0], current_position[1]+1)
        # check diagonally left-down
        elif current_position[0] > 408 and grid[current_position[0]-1].loc[current_position[1]+1] == 0:
            current_position = (current_position[0]-1, current_position[1]+1)
        # check diagonally right-down
        elif current_position[0] < 518 and grid[current_position[0]+1].loc[current_position[1]+1] == 0:
            current_position = (current_position[0]+1, current_position[1]+1)
        else:
            grid[current_position[0]].loc[current_position[1]] = 2
            sand_rest_counter += 1
            sand_rest = True

print(f"sand rest counter: {sand_rest_counter}")

sand rest counter: 888


In [18]:
# part 2 - reset the grid, add a bottom layer, and continue adding sand until it
# comes to rest at the origin point - (500, 0)
# assuming the pattern is set up such that it's not going to overflow to infinity on the left side, let's just make a really big grid

grid = pd.DataFrame(np.zeros(shape=(169,1000), dtype=int), index=list(range(169)), columns=list(range(1000)))

# now draw the rock lines -- set them to 1
for rp in rock_points:
    for i in range(len(rp) - 1):
        point_a = rp[i]
        point_b = rp[i+1]
        
        # if the "right" index (column numbers) are same
        if point_a[0] == point_b[0]:
            start_point = min(point_a[1], point_b[1])
            end_point = max(point_a[1], point_b[1])
            grid[point_a[0]].loc[start_point:end_point] = 1
        
        # if the "down" index (row numbers) are same
        if point_a[1] == point_b[1]:
            start_point = min(point_a[0], point_b[0])
            end_point = max(point_a[0], point_b[0])
            grid.loc[point_a[1]].loc[start_point:end_point] = 1

# draw the bottom line
grid.loc[168] = 1

# now simulate the sand -- our exit condition and boundary conditions change

master_exit = False
sand_rest_counter = 0

while not master_exit:
    sand_rest = False
    current_position = (500,0)
    while not sand_rest:
        # check if reached bottom
        if grid[500].loc[1] != 0 and grid[501].loc[1] != 0 and grid[499].loc[1] != 0:
            sand_rest_counter += 1
            master_exit = True
            break
        # check directly below
        elif grid[current_position[0]].loc[current_position[1]+1] == 0:
            current_position = (current_position[0], current_position[1]+1)
        # check diagonally left-down
        elif current_position[0] > 0 and grid[current_position[0]-1].loc[current_position[1]+1] == 0:
            current_position = (current_position[0]-1, current_position[1]+1)
        # check diagonally right-down
        elif current_position[0] < 999 and grid[current_position[0]+1].loc[current_position[1]+1] == 0:
            current_position = (current_position[0]+1, current_position[1]+1)
        else:
            grid[current_position[0]].loc[current_position[1]] = 2
            sand_rest_counter += 1
            sand_rest = True

print(f"sand rest counter: {sand_rest_counter}")

sand rest counter: 26461


In [ ]:
# this was inefficient - took a long time to run. got the job done though.
# it can probably be made better by skipping steps when the sand is falling.